In [1]:
import sys  
sys.path.insert(0, '../py')
from graviti import *

from numpy.linalg import norm
import numpy as np
import os
import os.path
from os import path
import sys
import glob
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
import hdbscan
import pandas as pd
import umap
import networkx as nx
from scipy import sparse, linalg
import pickle
from sklearn.preprocessing import normalize, scale
from scipy.sparse import find
from numpy.linalg import norm
import timeit
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [46]:
dirname = '../h5/id_52/' # the path to *features.npz files 
#dirname = '../data/id_52/' # the path to *features.npz files 
sample = '52' #sys.argv[2]  # the sample id
size = 500000 # number of nuclei, use negative value for full set
nn = 10 # set the number of nearest neighbor in the umap-graph. Will be used in CovD as well

features = ['area',
            'perimeter',
            'solidity',
            'eccentricity',
            'circularity',
            'mean_intensity',
            'std_intensity',
            #'cov_intensity',
           ]

In [3]:
counter = 0
for f in glob.glob(dirname+'/*features.npz'): # for every fov
    counter += 1
    if counter == 1:            # set up the data arrays
        data = np.load(f,allow_pickle=True)
        fov = data['fov']
        xy = data['centroids']
        morphology = data['morphology']
    else:                       # update the data arrays
        data = np.load(f,allow_pickle=True)
        fov = np.vstack((fov,data['fov']))
        xy = np.vstack((xy, data['centroids']))
        morphology = np.vstack((morphology, data['morphology']))

# Create dataframes with spatial and morphological measurements
df_fov = pd.DataFrame(data=fov, columns=['fov_row','fov_col']) # field of view dataframe
df_xy = pd.DataFrame(data=xy, columns=['cx','cy'])   # centroid dataframe
df_morphology = pd.DataFrame(data=morphology, columns=['area','perimeter','solidity','eccentricity','circularity','mean_intensity','std_intensity'])

# Concatenate spatial and morphological dataframes
df = pd.concat([df_fov,df_xy, df_morphology],axis=1)

In [47]:
# filter by percentiles in morphologies (hardcoded in function filtering) and introduce coeff. of var
if size == 0:
    fdf = filtering(df) # filter out extremes in morphology
else:
    fdf = filtering(df).sample(n=size) # filter out morphological outlyers and subsample nuclei

pos = fdf[fdf.columns[2:4]].to_numpy() # Get the positions of centroids 

# Building the UMAP graph
#filename = '../py/ID'+str(sample)+'.size'+str(size)+'.nn'+str(nn)+'.graph.npz' # the adj sparse matrix
print('Creating the graph')
A = space2graph(pos,nn)
#sparse.save_npz(filename, A)

Creating the graph


In [48]:
# Generation of the nn of the sampled nodes
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors
X = df[df.columns[2:4]].to_numpy() # the full array of position
n_neighbors = df.shape[0]//size + 10
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree',n_jobs=-1).fit(X) 
#distances, indices = nbrs.kneighbors(X[fdf.index,:]) # the knn from the sampled nodes
distances, indices = nbrs.kneighbors(X) # the knn from the sampled nodes

In [49]:
# Parallel generation of the local covd
data = scale(df[features].to_numpy(), with_mean=False) #get the morphological data and rescale the data by std 

# Get numb of cores
num_cores = multiprocessing.cpu_count() # numb of cores
    
#filename = '../py/ID'+str(sample)+'.size'+str(size)+'.nn'+str(nn)+'.descriptor.pickle'    # the descriptor
print('Generating the descriptor')
processed_list = Parallel(n_jobs=num_cores)(delayed(covd_parallel_sparse)(node,data,indices) 
                                                            for node in tqdm(list(fdf.index))
                                                   )





  0%|          | 0/500000 [00:00<?, ?it/s]

Generating the descriptor






  0%|          | 4/500000 [00:00<7:50:01, 17.73it/s]



  0%|          | 8/500000 [00:06<66:21:36,  2.09it/s]



  0%|          | 12/500000 [00:06<47:59:24,  2.89it/s]



  0%|          | 36/500000 [00:06<33:46:45,  4.11it/s]



  0%|          | 72/500000 [00:06<23:46:04,  5.84it/s]



  0%|          | 280/500000 [00:06<16:39:11,  8.34it/s]



  0%|          | 792/500000 [00:06<11:39:25, 11.90it/s]



  0%|          | 1924/500000 [00:06<8:08:42, 16.99it/s]



  1%|          | 2778/500000 [00:06<5:41:48, 24.24it/s]



  1%|          | 4120/500000 [00:07<3:58:52, 34.60it/s]



  1%|          | 6168/500000 [00:07<2:46:41, 49.38it/s]



  2%|▏         | 8216/500000 [00:07<1:56:27, 70.38it/s]



  2%|▏         | 10264/500000 [00:07<1:21:24, 100.26it/s]



  2%|▏         | 12312/500000 [00:07<56:59, 142.60it/s]  



  3%|▎         | 14360/500000 [00:08<40:01, 202.20it/s]



  3%|▎         | 16408/500000 [00:08<28:09, 286.28it/s]



  4%|▎         | 18456/500000 [00:08<19:51, 404.22it/s]


 42%|████▏     | 210968/500000 [00:30<00:31, 9128.74it/s]



 43%|████▎     | 213016/500000 [00:30<00:31, 9013.57it/s]



 43%|████▎     | 215064/500000 [00:30<00:30, 9497.01it/s]



 43%|████▎     | 217112/500000 [00:30<00:29, 9632.10it/s]



 44%|████▍     | 219160/500000 [00:31<00:28, 9766.07it/s]



 44%|████▍     | 221208/500000 [00:31<00:29, 9336.71it/s]



 45%|████▍     | 223256/500000 [00:31<00:29, 9385.00it/s]



 45%|████▌     | 225304/500000 [00:31<00:28, 9797.38it/s]



 45%|████▌     | 227352/500000 [00:31<00:28, 9589.49it/s]



 46%|████▌     | 229400/500000 [00:32<00:27, 9722.23it/s]



 46%|████▋     | 231448/500000 [00:32<00:27, 9850.68it/s]



 47%|████▋     | 233496/500000 [00:32<00:26, 9907.72it/s]



 47%|████▋     | 235544/500000 [00:32<00:26, 10019.08it/s]



 48%|████▊     | 237592/500000 [00:32<00:25, 10124.77it/s]



 48%|████▊     | 239640/500000 [00:33<00:26, 9705.03it/s] 



 48%|████▊     | 241688/500000 [00:33<00:26, 9822.56it/s]



 49%|████▊     | 2426

 87%|████████▋ | 434200/500000 [00:53<00:07, 9328.10it/s]



 87%|████████▋ | 436248/500000 [00:54<00:06, 9389.71it/s]



 88%|████████▊ | 438296/500000 [00:54<00:06, 9455.12it/s]



 88%|████████▊ | 440344/500000 [00:54<00:06, 9651.45it/s]



 88%|████████▊ | 442392/500000 [00:54<00:05, 9936.83it/s]



 89%|████████▉ | 444440/500000 [00:54<00:05, 9802.18it/s]



 89%|████████▉ | 446488/500000 [00:55<00:05, 9839.26it/s]



 90%|████████▉ | 448536/500000 [00:55<00:05, 9756.61it/s]



 90%|█████████ | 450584/500000 [00:55<00:05, 9687.51it/s]



 91%|█████████ | 452632/500000 [00:55<00:04, 9920.37it/s]



 91%|█████████ | 454680/500000 [00:55<00:04, 9589.97it/s]



 91%|█████████▏| 456728/500000 [00:56<00:04, 9703.58it/s]



 92%|█████████▏| 458776/500000 [00:56<00:05, 7958.32it/s]



 92%|█████████▏| 460824/500000 [00:56<00:04, 8709.67it/s]



 93%|█████████▎| 462872/500000 [00:56<00:04, 9171.20it/s]



 93%|█████████▎| 464920/500000 [00:57<00:03, 8938.97it/s]



 93%|█████████▎| 466968/

In [50]:
# Construct the descriptor array
descriptor = np.zeros((len(processed_list),processed_list[0][1].shape[0]))
for r in range(len(processed_list)):
    descriptor[r,:] = processed_list[r][1] # covd descriptors of the connected nodes
#pickle.dump( descriptor, open( filename, "wb" ) )

In [ ]:
# Get info about the graph
row_idx, col_idx, values = find(A) #A.nonzero() # nonzero entries
print('Generating the diversity index')
fdf['diversity'] = Parallel(n_jobs=num_cores)(delayed(covd_gradient_parallel)(node,
                                                                          descriptor,
                                                                          row_idx,col_idx,values) 
                               for node in tqdm(range(descriptor.shape[0])))





  0%|          | 0/500000 [00:00<?, ?it/s]

Generating the diversity index






  0%|          | 1/500000 [00:00<103:16:57,  1.34it/s]



  0%|          | 4/500000 [00:00<75:40:18,  1.84it/s] 



  0%|          | 8/500000 [00:06<113:49:33,  1.22it/s]



  0%|          | 12/500000 [00:07<81:47:28,  1.70it/s]



  0%|          | 16/500000 [00:07<58:19:18,  2.38it/s]



  0%|          | 28/500000 [00:07<41:17:40,  3.36it/s]



  0%|          | 40/500000 [00:07<29:18:26,  4.74it/s]



  0%|          | 72/500000 [00:07<20:39:56,  6.72it/s]



  0%|          | 120/500000 [00:07<14:36:24,  9.51it/s]



  0%|          | 152/500000 [00:07<10:24:41, 13.34it/s]



  0%|          | 184/500000 [00:07<7:25:19, 18.71it/s] 



  0%|          | 216/500000 [00:08<5:21:21, 25.92it/s]



  0%|          | 248/500000 [00:08<3:55:56, 35.30it/s]



  0%|          | 312/500000 [00:08<2:53:34, 47.98it/s]



  0%|          | 376/500000 [00:08<2:11:06, 63.51it/s]



  0%|          | 408/500000 [00:08<1:40:10, 83.11it/s]



  0%|          | 472/500000 [00:08<1:17:32, 107.38it/s]



  0%| 

  1%|▏         | 6968/500000 [00:30<28:05, 292.55it/s]



  1%|▏         | 7032/500000 [00:30<26:54, 305.43it/s]



  1%|▏         | 7096/500000 [00:30<26:39, 308.18it/s]



  1%|▏         | 7160/500000 [00:31<26:22, 311.34it/s]



  1%|▏         | 7224/500000 [00:31<26:19, 312.05it/s]



  1%|▏         | 7288/500000 [00:31<27:30, 298.54it/s]



  1%|▏         | 7352/500000 [00:31<26:29, 310.00it/s]



  1%|▏         | 7416/500000 [00:31<25:35, 320.76it/s]



  1%|▏         | 7480/500000 [00:32<25:38, 320.23it/s]



  2%|▏         | 7544/500000 [00:32<27:47, 295.33it/s]



  2%|▏         | 7608/500000 [00:32<27:27, 298.93it/s]



  2%|▏         | 7672/500000 [00:32<25:59, 315.76it/s]



  2%|▏         | 7736/500000 [00:33<25:52, 317.04it/s]



  2%|▏         | 7800/500000 [00:33<26:06, 314.18it/s]



  2%|▏         | 7864/500000 [00:33<25:39, 319.70it/s]



  2%|▏         | 7928/500000 [00:33<25:47, 317.91it/s]



  2%|▏         | 7992/500000 [00:33<25:17, 324.31it/s]



  2%|▏        

  3%|▎         | 14392/500000 [00:54<26:10, 309.24it/s]



  3%|▎         | 14456/500000 [00:54<25:22, 319.02it/s]



  3%|▎         | 14520/500000 [00:54<24:53, 324.96it/s]



  3%|▎         | 14584/500000 [00:55<26:02, 310.74it/s]



  3%|▎         | 14648/500000 [00:55<26:16, 307.81it/s]



  3%|▎         | 14712/500000 [00:55<27:09, 297.77it/s]



  3%|▎         | 14776/500000 [00:55<27:14, 296.90it/s]



  3%|▎         | 14840/500000 [00:56<26:21, 306.82it/s]



  3%|▎         | 14904/500000 [00:56<25:33, 316.40it/s]



  3%|▎         | 14968/500000 [00:56<25:45, 313.74it/s]



  3%|▎         | 15032/500000 [00:56<25:45, 313.84it/s]



  3%|▎         | 15096/500000 [00:56<26:03, 310.16it/s]



  3%|▎         | 15160/500000 [00:57<25:39, 314.92it/s]



  3%|▎         | 15224/500000 [00:57<24:55, 324.13it/s]



  3%|▎         | 15288/500000 [00:57<26:34, 303.98it/s]



  3%|▎         | 15352/500000 [00:57<26:04, 309.77it/s]



  3%|▎         | 15416/500000 [00:57<25:18, 319.09it/s]


...

 47%|████▋     | 232824/500000 [12:43<14:37, 304.46it/s]



 47%|████▋     | 232888/500000 [12:43<14:29, 307.26it/s]



 47%|████▋     | 232952/500000 [12:43<15:18, 290.72it/s]



 47%|████▋     | 233016/500000 [12:44<15:28, 287.47it/s]



 47%|████▋     | 233080/500000 [12:44<14:59, 296.70it/s]



 47%|████▋     | 233144/500000 [12:44<15:10, 293.20it/s]



 47%|████▋     | 233208/500000 [12:44<14:19, 310.41it/s]



 47%|████▋     | 233272/500000 [12:44<14:38, 303.53it/s]



 47%|████▋     | 233336/500000 [12:45<14:10, 313.72it/s]



 47%|████▋     | 233400/500000 [12:45<14:00, 317.37it/s]



 47%|████▋     | 233464/500000 [12:45<14:17, 310.79it/s]



 47%|████▋     | 233528/500000 [12:45<15:00, 295.92it/s]



 47%|████▋     | 233592/500000 [12:45<14:54, 297.77it/s]



 47%|████▋     | 233656/500000 [12:46<14:40, 302.45it/s]



 47%|████▋     | 233720/500000 [12:46<14:30, 305.81it/s]



 47%|████▋     | 233784/500000 [12:46<14:12, 312.43it/s]



 47%|████▋     | 233848/500000 [12:46<14

 48%|████▊     | 240056/500000 [13:06<14:52, 291.13it/s]



 48%|████▊     | 240120/500000 [13:07<14:58, 289.25it/s]



 48%|████▊     | 240184/500000 [13:07<14:07, 306.41it/s]



 48%|████▊     | 240248/500000 [13:07<13:43, 315.55it/s]



 48%|████▊     | 240312/500000 [13:07<13:09, 328.90it/s]



 48%|████▊     | 240376/500000 [13:07<13:16, 325.99it/s]



 48%|████▊     | 240440/500000 [13:08<13:49, 312.77it/s]



 48%|████▊     | 240504/500000 [13:08<14:17, 302.58it/s]



 48%|████▊     | 240568/500000 [13:08<13:30, 320.20it/s]



 48%|████▊     | 240632/500000 [13:08<13:43, 314.82it/s]



 48%|████▊     | 240696/500000 [13:08<13:49, 312.61it/s]



 48%|████▊     | 240760/500000 [13:09<13:43, 314.80it/s]



 48%|████▊     | 240824/500000 [13:09<13:58, 309.02it/s]



 48%|████▊     | 240888/500000 [13:09<14:33, 296.57it/s]



 48%|████▊     | 240952/500000 [13:09<13:35, 317.67it/s]



 48%|████▊     | 241016/500000 [13:09<12:54, 334.35it/s]



 48%|████▊     | 241080/500000 [13:10<14

 49%|████▉     | 247288/500000 [13:30<14:51, 283.33it/s]



 49%|████▉     | 247352/500000 [13:30<15:03, 279.77it/s]



 49%|████▉     | 247416/500000 [13:31<14:57, 281.54it/s]



 49%|████▉     | 247480/500000 [13:31<14:34, 288.90it/s]



 50%|████▉     | 247544/500000 [13:31<13:54, 302.65it/s]



 50%|████▉     | 247608/500000 [13:31<13:31, 311.20it/s]



 50%|████▉     | 247672/500000 [13:31<13:09, 319.46it/s]



 50%|████▉     | 247736/500000 [13:32<13:14, 317.50it/s]



 50%|████▉     | 247800/500000 [13:32<12:48, 328.27it/s]



 50%|████▉     | 247864/500000 [13:32<12:36, 333.24it/s]



 50%|████▉     | 247928/500000 [13:32<13:31, 310.60it/s]



 50%|████▉     | 247992/500000 [13:32<13:52, 302.69it/s]



 50%|████▉     | 248056/500000 [13:33<13:48, 304.18it/s]



 50%|████▉     | 248120/500000 [13:33<13:31, 310.53it/s]



 50%|████▉     | 248184/500000 [13:33<13:42, 306.10it/s]



 50%|████▉     | 248248/500000 [13:33<13:20, 314.60it/s]



 50%|████▉     | 248312/500000 [13:33<12

 51%|█████     | 254520/500000 [13:54<12:59, 315.02it/s]



 51%|█████     | 254584/500000 [13:54<13:19, 307.11it/s]



 51%|█████     | 254648/500000 [13:54<13:45, 297.14it/s]



 51%|█████     | 254712/500000 [13:55<13:42, 298.30it/s]



 51%|█████     | 254776/500000 [13:55<13:47, 296.42it/s]



 51%|█████     | 254840/500000 [13:55<13:28, 303.14it/s]



 51%|█████     | 254904/500000 [13:55<12:43, 320.96it/s]



 51%|█████     | 254968/500000 [13:55<12:28, 327.23it/s]



 51%|█████     | 255032/500000 [13:56<13:03, 312.81it/s]



 51%|█████     | 255096/500000 [13:56<12:18, 331.83it/s]



 51%|█████     | 255160/500000 [13:56<12:43, 320.84it/s]



 51%|█████     | 255224/500000 [13:56<12:42, 320.92it/s]



 51%|█████     | 255288/500000 [13:56<13:10, 309.50it/s]



 51%|█████     | 255352/500000 [13:57<13:16, 307.12it/s]



 51%|█████     | 255416/500000 [13:57<13:38, 298.69it/s]



 51%|█████     | 255480/500000 [13:57<12:59, 313.79it/s]



 51%|█████     | 255544/500000 [13:57<12

 52%|█████▏    | 261752/500000 [14:17<14:04, 282.13it/s]



 52%|█████▏    | 261816/500000 [14:17<13:25, 295.75it/s]



 52%|█████▏    | 261880/500000 [14:18<13:24, 295.88it/s]



 52%|█████▏    | 261944/500000 [14:18<12:43, 311.62it/s]



 52%|█████▏    | 262008/500000 [14:18<13:10, 301.08it/s]



 52%|█████▏    | 262072/500000 [14:18<12:43, 311.71it/s]



 52%|█████▏    | 262136/500000 [14:18<12:45, 310.62it/s]



 52%|█████▏    | 262200/500000 [14:19<12:55, 306.83it/s]



 52%|█████▏    | 262264/500000 [14:19<12:14, 323.75it/s]



 52%|█████▏    | 262328/500000 [14:19<11:49, 334.88it/s]



 52%|█████▏    | 262392/500000 [14:19<11:59, 330.47it/s]



 52%|█████▏    | 262456/500000 [14:19<12:15, 323.00it/s]



 53%|█████▎    | 262520/500000 [14:20<13:28, 293.76it/s]



 53%|█████▎    | 262584/500000 [14:20<13:03, 302.92it/s]



 53%|█████▎    | 262648/500000 [14:20<12:29, 316.57it/s]



 53%|█████▎    | 262712/500000 [14:20<12:43, 310.62it/s]



 53%|█████▎    | 262776/500000 [14:20<12

In [ ]:
N = 50 # number of linear bins for the contour visualization

# Contour visualization
fdf['x_bin'] = pd.cut(fdf['cx'], N, labels=False) # define the x bin label
fdf['y_bin'] = pd.cut(fdf['cy'], N, labels=False) # define the y bin label

# define the pivot tabel for the contour plot
table = pd.pivot_table(fdf, 
                       values='diversity', 
                       index=['x_bin'],
                       columns=['y_bin'],
                       aggfunc=np.sum, # take the mean of the entries in the bin
                       fill_value=None)

X=table.columns.values
Y=table.index.values
Z=np.log10(table.values)
Xi,Yi = np.meshgrid(X, Y)

fig, ax = plt.subplots(figsize=(10,10))
cs = ax.contourf(Yi, Xi, Z, 
                 alpha=1.0, 
                 levels=10,
                 cmap=plt.cm.viridis);
cbar = fig.colorbar(cs)
plt.savefig('test_sum.png')

In [ ]:
 plotlyContourPlot(fdf,filename)

In [ ]:
fdf['field'].plot.hist(bins=N)
plt.show()

In [ ]:
count_nuclei = fdf.groupby(['x_bin', 'y_bin']).size().to_numpy()

variability = fdf.groupby(['x_bin', 'y_bin'])['field'].sum().to_numpy()

test_df = pd.DataFrame(list(zip(count_nuclei, variability)),
                  columns=['count','variability'])

In [ ]:
print(test_df.corr(method='pearson'))
test_df.plot.scatter(x='count',y='variability')

In [ ]:
fdf.groupby(['x_bin', 'y_bin']).size().plot.hist(bins=N//10)

In [ ]:
fdf.groupby(['x_bin', 'y_bin'])['field'].mean().plot.hist(bins=N//10)

In [ ]:
import plotly.express as px
fig = px.scatter(fdf, 
                 x="x_bin", 
                 y="y_bin",
                 color="field",
                 #size=1,
                 opacity=0.5,
                 color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(template='simple_white',
                 height=800, width=800)

fig.show()

In [ ]:
# Build figure
fig = go.Figure()

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=fdf['x_bin'],
        y=fdf['y_bin'],
        marker=dict(
            color=fdf['field'],
            size=fdf['field'],
            opacity=0.5,
        ),
        showlegend=False
    )
)
fig.update_layout(template='simple_white',
                 height=500, width=500)
fig.show()